In [1]:
import pandas as pd
import collections
import os

In [2]:
df = pd.read_excel(io = 'data/export_dashboard_x_uk_x_filter_nativeretweets_geocode_55_378051_3_43597299999999_750km_2016_04_21_10_32_03.xlsx',
                   sheet_name = 'Stream')

In [3]:
df = df[df['Tweet content'].str.contains(r'#[A-Za-z0-9]*')]

In [4]:
import re
dictonary = collections.OrderedDict()
for lattitude, longitude, text in zip(list(df['Latitude']),
                                      list(df['Longitude']),
                                      list(df['Tweet content'])):
    hashtags = re.findall(r'\B#\w+', text)
    for hashtag in hashtags:
        if not hashtag in dictonary:
            dictonary[hashtag] = 0
        dictonary[hashtag] += 1

In [5]:
newDict = collections.OrderedDict()
# Iterate over all the items in dictionary and filter items which has even keys
for (key, value) in dictonary.items():
   # Check if key is even then add pair to new dictionary
   if value > 150:
       newDict[key] = value
 

In [6]:
sorted_dict = sorted(newDict.items(), key=lambda kv: kv[1], reverse=True)

In [7]:
import folium
from folium import plugins
from folium.plugins import HeatMap

/usr/local/lib/python2.7/dist-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [8]:
def getDataByIndex(sorted_dict, df, index):
    hashtag_df = df[df['Tweet content'].str.contains(sorted_dict[0][0])]
    bestdict = {'latitude': [], 'longitude':[],  'count': []}
    for  lattitude, longitude, text in zip(list(df['Latitude']),
                                          list(df['Longitude']),
                                          list(df['Tweet content'])):
        if text.count(sorted_dict[index][0]) > 0:
            bestdict['latitude'].append(lattitude)
            bestdict['longitude'].append(longitude)
            bestdict['count'].append(text.count(sorted_dict[index][0]))
    return pd.DataFrame(bestdict)

In [9]:
def generateBaseMap(default_location=[52.218258, -3.541508], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [10]:
def getHeatmap(newMap, data_frame, hashtag):
    
    return HeatMap(data=data_frame[['latitude', 'longitude', 'count']]
        .groupby(['latitude', 'longitude']).sum()
        .reset_index().values.tolist(),radius=8,
        max_zoom=13, name=hashtag)

In [11]:
m = generateBaseMap()
for i in range(20):
    hashtagDF = getDataByIndex(sorted_dict, df, i)
    heatmap = getHeatmap(m, hashtagDF, sorted_dict[i][0])
    m.add_child(heatmap)

folium.LayerControl().add_to(m)

In [12]:
m